# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
# from unityagents import UnityEnvironment
# import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
PATH = "/Volumes/BC_Clutch/Dropbox/Programming/Classes/Udacity/DeepRLND/rl_continuous_control/"

In [3]:
# fp = PATH + "data/Reacher20.app"
# env = UnityEnvironment(file_name=fp)

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
# brain_name = env.brain_names[0]
# brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# # reset the environment
# env_info = env.reset(train_mode=True)[brain_name]

# # number of agents
# num_agents = len(env_info.agents)
# print('Number of agents:', num_agents)

# # size of each action
# action_size = brain.vector_action_space_size
# print('Size of each action:', action_size)

# # examine the state space 
# states = env_info.vector_observations
# state_size = states.shape[1]
# print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
# print('The state for the first agent looks like:', states[0])

In [ ]:
import re 
import time
import datetime
import numpy as np

from agents.DDPG import DDPG
from util import *

# Number of agents to train.  Must be "single" or "multi."
# BUG cannot run more than one unity environment in same loop (so cannot be "both")
TRAIN_MODE = "multi"

env_dict = {
            "Reacher20":["unity","Reacher20.app","multi",30.0], # 30.0
#             "Reacher20":["unity","Reacher_Linux_NoVis2/Reacher.x86_64","multi",30.0],
            "Reacher1":["unity","Reacher1.app","single",30.0], # 30.0
#             "Reacher1":["unity","Reacher_Linux_NoVis1/Reacher.x86_64","single",30.0],
            "Pendulum":["gym","Pendulum-v0","single",2000.0],
            "BipedalWalker":["gym","BipedalWalker-v2","single",300.0] # 300.0
            }

# BUG cannot run more than one agent in a single unity environment in same loop
# per open issue at https://github.com/Unity-Technologies/ml-agents/issues/1167
agent_dict = {
              "DDPG":[DDPG,"both"],
              # "DDPGplus":[DDPGplus,"both"],
              # "D4PG":[D4PG,"single"],
              # "TD3":[TD3,"both"]
             }

results = train_envs(PATH, env_dict, agent_dict,TRAIN_MODE)
print(results)

Train Mode: Multi-Agent
Environment: Reacher20	Policy: DDPG


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Number of agents: 20
There are 20 agents.  Each observes a state with length 33
The state for the first agent looks like:
[ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]
Size of each action: 4
Episode 20	Score TAS/Mean/Max/Min: 0.05/0.08/0.25/0.00	00:57:26 
Episode 40	Score TAS/Mean/Max/Min: 0.46/1.38/2.92/0.39	02:01:07 
Episode 60	Score TAS/Mean/Max/Min: 0.90/1.86/5.21/0.43	03:11:27 
Episode 80	Score TAS/Mean/Max/Min: 1.34/3.32/7.19/0.00	04:23:28  
Episode 100	Score TAS/Mean/Max/Min: 1.90/5.40/9.33/0.31	

In [ ]:
# train on Udacity GPUs
# check agent (DDPG?) to troubleshoot